<a href="https://colab.research.google.com/github/hksantosh/sscm/blob/main/ASCP/Production_planning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ortools

In [ ]:
import numpy as np
import pandas as pd
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver.CreateSolver('SCIP')
infinity = solver.infinity()

# Formulating the Problem

## Indices
4 quarters - [1,2,3,4]


In [ ]:
quarters = [1,2,3,4]

## Parameters
*   Regular production costs \$20 per unit
*   Overtime production costs \$25 per unit
*   Subcontracting costs \$28 per unit
*   Inventory costs \$3 per unit-period
*   Inventory at beginning of the year = 300 units
*   Inventory at end of the year = 300 units


In [ ]:
regular_prod_cost = 20
overtime_prod_cost = 25
subcontract_cost = 28
inventory_cost = 3
starting_inventory = 300
year_end_inventory = 300

qrtrly_demand = [900,1500,1600,3000]
qrtrly_regular_capacity = [1000,1200,1300,1300]
qrtrly_overtime_capacity = [100,150,200,200]
qrtrly_subcontract_capacity = [500,500,500,500]

## Decision Variables
* Units produced in each quarter using regular production - $R_{1}, R_{2}, R_{3}, R_{4}$
* Units produced in each quarter using Overtime - $V_{1}, V_{2}, V_{3}, V_{4}$
* Units produced in each quarter using Subcontracts - $S_{1}, S_{2}, S_{3}, S_{4}$

## Other Variables
* Inventory at the start of each quarter - $B_{1}, B_{2}, B_{3}, B_{4}$
* Inventory at the end of each quarter - $E_{1}, E_{2}, E_{3}, E_{4}$

In [ ]:
df = pd.DataFrame(index=quarters,columns=['regular','overtime','subcontract','inventory'])

for i in quarters:
  for j in ['regular','overtime','subcontract']:
    df.at[i,j] = solver.IntVar(0, infinity, 'X[%d][%s]' %(i,j))

for i in quarters:
  if(i==1):
    inventory = starting_inventory
  else:
    inventory = df.at[i-1,'inventory'] + df.at[i-1,'regular'] + df.at[i-1,'overtime'] + df.at[i-1,'subcontract'] 
                  - qrtrly_demand[i-2] # i-2 as this is an array & index starts at 0 whereas our dataframe row index starts at 1 (quarter no.)

  df.at[i,'inventory'] = solver.IntVar(inventory, inventory, 'X[%d][inventory]' %(i))

## Constraints
* Demand constraint for each quarter - 900, 1500, 1600, 3000
* Regular production capacity constraint for each quarter - 1000, 1200, 1300, 1300
* Overtime production capacity constraint for each quarter - 100, 150, 200, 200
*	Subcontract production capacity constraint for each quarter - 500, 500, 500, 500

In [ ]:
#Create the constraints

for i in quarters:
  ## Set capacity constraints
  solver.Add(df.at[i,'regular'] <= qrtrly_regular_capacity[i-1])
  solver.Add(df.at[i,'overtime'] <= qrtrly_overtime_capacity[i-1])
  solver.Add(df.at[i,'subcontract'] <= qrtrly_subcontract_capacity[i-1])

  # Set demand satisfaction constraint
  solver.Add(df.at[i,'inventory'] >= 0

## Set year end inventory constraint (in other words end of q4 inventory constraint)
solver.Add( (df.at[4,'inventory'] + df.at[4,'regular'] + df.at[4,'overtime'] + df.at[4,'subcontract'] - qrtrly_demand[3]) == year_end_inventory )

## Objective Function
Minimize the total production cost = $\min( \sum_{q=1}^{4} (20R_{q}+25V_{q}+28S_{q}+3I_{q})$\)


In [ ]:
objective_terms = []
for i in quarters:
  objective_terms.append(df.at[i,'regular']*regular_prod_cost)
  objective_terms.append(df.at[i,'overtime']*overtime_prod_cost)
  objective_terms.append(df.at[i,'subcontract']*subcontract_cost)
  objective_terms.append(df.at[i,'inventory']*inventory_cost)

solver.Minimize(solver.Sum(objective_terms))
status = solver.Solve()